# Theta-pinch startup
In the Grad Shafranov workbook, we saw how closed magnetic flux surfaces are created by magnetic field coils outside the plasma. In this example, we see how currents inside the plasma can cause the same thing.

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150  
plt.ioff()
%matplotlib widget
na = np.newaxis

In [ ]:
# Set mu0
u0 = 4 * np.pi * 10**-7

In [ ]:
# Setup simulation domain
simulation_y = 45
simulation_x = 200
X = np.arange(simulation_x)[na, :] * np.ones((simulation_y, simulation_x))
Y = np.arange(simulation_y)[:, na] * np.ones((simulation_y, simulation_x))

In [ ]:
# Initialize matrices describing the current and magnetic field in the plasma
currenti = np.zeros((simulation_y, simulation_x))
B = np.zeros((simulation_y, simulation_x))

# Initial conditions
currenti[-1, :] = 1200
currenti[-1, :int(simulation_x / 4)] += 5000 * np.linspace(1, 0, int(simulation_x / 4))
currenti[-1, -int(simulation_x / 4):] += 5000 * np.linspace(0, 1, int(simulation_x / 4))
currenti[0, :] = -1200
currenti[0, :int(simulation_x / 4)] += -5000 * np.linspace(1, 0, int(simulation_x / 4))
currenti[0, -int(simulation_x / 4):] += -5000 * np.linspace(0, 1, int(simulation_x / 4))

# Specify where the current is in the plasma
beam_current = ((((simulation_x / 60) - np.minimum(simulation_x / 60, np.abs(X - (14.7 * simulation_x / 31)))) * \
               ((simulation_y / 90) - np.minimum(simulation_y / 90, np.abs(Y - (simulation_y / 4)))))) - \
               ((((simulation_x / 60) - np.minimum(simulation_x / 60, np.abs(X - (14.7 * simulation_x / 31)))) * \
               ((simulation_y / 90) - np.minimum(simulation_y / 90, np.abs(Y - (3 * simulation_y / 4)))))) + \
               ((((simulation_x / 60) - np.minimum(simulation_x / 60, np.abs(X - (16.3 * simulation_x / 31)))) * \
               ((simulation_y / 90) - np.minimum(simulation_y / 90, np.abs(Y - (simulation_y / 4)))))) - \
               ((((simulation_x / 60) - np.minimum(simulation_x / 60, np.abs(X - (16.3 * simulation_x / 31)))) * \
               ((simulation_y / 90) - np.minimum(simulation_y / 90, np.abs(Y - (3 * simulation_y / 4))))))
    
beam_current = beam_current / beam_current.max()

# Plot to look at what the current in the plasma will look like
f, ax = plt.subplots()
cont = ax.contourf(X, Y, beam_current)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('Current (normalized)')
plt.colorbar(cont)
plt.show()

Now that we have the simulation domain set up and have dictated what we want our current profile to look like, let's conduct a simulation in which the current in the plasma starts at 0 and slowly increases.

In [ ]:
# Set ranges for plotting later
current = currenti * 10000
levels = np.logspace(np.log(0.001), np.log(50), 25)
levels = np.hstack((-levels[::-1], levels))


# Iterate
niter = 100
B_iterations = np.zeros((simulation_y, simulation_x, niter))

for i in range(niter):

    # Calculate distances between points where there's current and everywhere we want to compute B
    # (for Biot-Savart)
    axs = X[np.abs(current) > 0].astype(int)
    ays = Y[np.abs(current) > 0].astype(int)
    adxs = np.subtract.outer(X, axs) * 4 / 100
    adys = np.subtract.outer(Y, ays) * 4 / 100
    ads = np.maximum(1e-6, np.sqrt(adxs**2 + adys**2))
    
    # Calculate B
    B_iterations[:, :, i] = -u0 * np.sum(current[ays, axs][na, na, :] / ads, axis=2) / 4 / np.pi

    # Turn up the current based on the iteration number
    current = currenti + (np.minimum(100, i) * beam_current * 70)
    if i > 100:
        current = current + (currenti * (i - 100) * 0.04)
    

In [ ]:
# Replay simulation
fig, ax = plt.subplots()

def animate(ti):
    plt.cla()
    plt.contour(np.linspace(-18, 18, 50),
                  np.linspace(-1.5, 1.5, 35),
                  B_iterations[5:40, 75:125, ti],
                  levels=np.linspace(-0.368, 0.368, 50)**3)
    plt.grid(True)

matplotlib.animation.FuncAnimation(fig, animate, frames=niter)